In [ ]:
# --- 1. SETUP & INSTALLATION ---
# Installs necessary libraries and sets up directory structure.

!pip install --upgrade --quiet \
    "git+https://github.com/huggingface/diffusers" \
    "gguf" \
    "accelerate" \
    "peft" \
    "ipywidgets"

import os
import shutil

# Define Paths
# We move outputs to a clean folder, separate from models
OUTPUT_DIR = "/kaggle/working/generated_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Environment Ready.")
print(f"📂 Images will be saved to: {OUTPUT_DIR}")

In [ ]:
# --- 2. LOAD MODEL & OPTIMIZE MEMORY ---
import torch
import gc
from huggingface_hub import hf_hub_download
from diffusers import ZImagePipeline, ZImageTransformer2DModel, GGUFQuantizationConfig

# Configuration
GGUF_REPO = "unsloth/Z-Image-Turbo-GGUF"
GGUF_FILE = "z-image-turbo-Q8_0.gguf"

print("⏳ Downloading Model (approx 7GB)...")
try:
    local_gguf_path = hf_hub_download(GGUF_REPO, GGUF_FILE)
    
    print("⏳ Loading Transformer...")
    transformer = ZImageTransformer2DModel.from_single_file(
        local_gguf_path,
        quantization_config=GGUFQuantizationConfig(compute_dtype=torch.bfloat16),
        dtype=torch.bfloat16,
    )

    print("⏳ Initializing Pipeline...")
    pipe = ZImagePipeline.from_pretrained(
        "Tongyi-MAI/Z-Image-Turbo",
        transformer=transformer,
        torch_dtype=torch.bfloat16,
    )

    # --- CRITICAL MEMORY OPTIMIZATIONS FOR P100 ---
    # 1. CPU Offload: Keeps text encoder in RAM until needed
    pipe.enable_model_cpu_offload()
    
    # 2. VAE Tiling: Process image in small overlapping squares (Saves ~3-4GB VRAM)
    pipe.vae.enable_tiling()
    
    # 3. VAE Slicing: Decode batches one by one (Saves VRAM)
    pipe.vae.enable_slicing()

    print("\n✅ Model Loaded Successfully.")
    print("🛡️ Memory Protection Enabled (Tiling + Slicing active).")

except Exception as e:
    print(f"\n❌ Error loading model: {e}")

In [ ]:
# --- 3. ROBUST STUDIO UI (V2) ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from datetime import datetime

# --- SAFE RESOLUTION PRESETS ---
# These are mathematically guaranteed to work (Divisible by 16)
RATIO_PRESETS = {
    "Landscape (16:9)": (1280, 720),
    "Portrait (9:16)": (720, 1280),
    "Square (1:1)": (1024, 1024),
    "Cinematic (21:9)": (1536, 640),
}

# --- WIDGETS ---
style = {'description_width': '80px'}
layout_full = widgets.Layout(width='98%')
layout_half = widgets.Layout(width='48%')

# Header
lbl_header = widgets.HTML("<h2>🎨 Z-Image Turbo Studio</h2>")

# 1. Prompt Area
txt_prompt = widgets.Textarea(
    placeholder='Describe your image... (e.g. detailed human heart, anatomical drawing, 8k, masterpiece)',
    description='Prompt:',
    rows=3,
    style=style,
    layout=layout_full
)

txt_negative = widgets.Textarea(
    value='low quality, bad anatomy, worst quality, text, watermark, blurry, ugly, deformed',
    placeholder='Things to exclude...',
    description='Negative:',
    rows=1,
    style=style,
    layout=layout_full
)

# 2. Controls
dd_ratio = widgets.Dropdown(
    options=list(RATIO_PRESETS.keys()),
    value='Landscape (16:9)',
    description='Ratio:',
    style=style,
    layout=layout_half
)

num_seed = widgets.IntText(
    value=-1,
    description='Seed (-1=Rnd):',
    style={'description_width': '100px'},
    layout=layout_half
)

slider_steps = widgets.IntSlider(
    value=10,       # 10 is a safe sweet spot for Turbo
    min=4, max=30,
    description='Steps:',
    style=style,
    layout=layout_half
)

slider_cfg = widgets.FloatSlider(
    value=2.0,
    min=1.0, max=10.0, step=0.5,
    description='CFG:',
    style=style,
    layout=layout_half
)

btn_gen = widgets.Button(
    description=' GENERATE',
    button_style='success',
    icon='bolt',
    layout=widgets.Layout(width='100%', height='50px')
)

out_log = widgets.Output()
out_img = widgets.Output()

# --- GENERATION LOGIC ---
def run_generation(b):
    with out_log:
        clear_output()
        
        # 1. Get Resolution DIRECTLY from Presets (Fixes the 100x100 bug)
        selected_ratio = dd_ratio.value
        w, h = RATIO_PRESETS[selected_ratio]
        
        # 2. Get Seed
        current_seed = num_seed.value
        if current_seed == -1:
            current_seed = random.randint(0, 2**32-1)
            
        prompt = txt_prompt.value
        if not prompt:
            print("⚠️ Prompt is empty! using default.")
            prompt = "A beautiful landscape"

        print(f"⚙️ Config: {w}x{h} ({selected_ratio})")
        print(f"⚡ Steps: {slider_steps.value} | CFG: {slider_cfg.value}")
        print(f"🌱 Seed: {current_seed}")
        print(f"📝 Prompt: {prompt[:80]}...")

        # 3. Clean VRAM
        gc.collect()
        torch.cuda.empty_cache()
        
        try:
            # 4. Run Pipeline
            generator = torch.Generator("cuda").manual_seed(current_seed)
            
            with torch.inference_mode():
                image = pipe(
                    prompt=prompt,
                    negative_prompt=txt_negative.value,
                    height=h,
                    width=w,
                    num_inference_steps=slider_steps.value,
                    guidance_scale=slider_cfg.value,
                    generator=generator
                ).images[0]
            
            # 5. Save & Display
            ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"Img_{ts}_{w}x{h}_s{current_seed}.png"
            save_path = os.path.join(OUTPUT_DIR, filename)
            image.save(save_path)
            
            print(f"✅ Saved: {filename}")
            
            with out_img:
                clear_output()
                display(image)

        except Exception as e:
            print(f"\n❌ Error: {e}")
            print(f"Diagnostic: Requested {w}x{h}. If this fails, restart kernel.")

btn_gen.on_click(run_generation)

# --- UI ASSEMBLY ---
ui_v2 = widgets.VBox([
    lbl_header,
    widgets.VBox([txt_prompt, txt_negative], layout=widgets.Layout(margin='0 0 10px 0')),
    widgets.HBox([dd_ratio, num_seed]),
    widgets.HBox([slider_steps, slider_cfg]),
    widgets.Box([btn_gen], layout=widgets.Layout(margin='20px 0')),
    out_log,
    out_img
])

display(ui_v2)